In [1]:
import pandas as pd
import numpy as np

In [2]:
datos = pd.read_csv('C:\master\dataset.csv')

In [3]:
datos['date'] = pd.to_datetime(datos['date'])

In [4]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
LE.fit(datos["Shift"])
list(LE.classes_)

['Afternoon', 'Morning', 'Night']

In [5]:
datos["Shift"] = LE.transform(datos["Shift"])
datos["Shift"].value_counts()

0    279847
1    277391
2    267349
Name: Shift, dtype: int64

In [6]:
datos.head()

,GridID,date,Shift,Accident,Longitude.grid,Latitude.grid
0,1,2010-10-08,2,0,2.08,41.41
1,1,2011-02-16,1,0,2.08,41.41
2,1,2014-05-31,2,0,2.08,41.41
3,1,2011-04-03,0,0,2.08,41.41
4,1,2013-02-20,1,0,2.08,41.41


In [7]:
from datetime import datetime, date, time, timedelta
import calendar

In [8]:
Week_Day = []
for fecha in datos['date']:
    dia_semana = datetime.weekday(fecha)+1
    Week_Day.append(dia_semana)
    
datos['Week.Day'] = Week_Day
del Week_Day

In [9]:
datos["Type.Day"] = datos["Week.Day"].apply(lambda x: 'workday' if x<6 else 'weekend')

In [10]:
datos.head()

,GridID,date,Shift,Accident,Longitude.grid,Latitude.grid,Week.Day,Type.Day
0,1,2010-10-08,2,0,2.08,41.41,5,workday
1,1,2011-02-16,1,0,2.08,41.41,3,workday
2,1,2014-05-31,2,0,2.08,41.41,6,weekend
3,1,2011-04-03,0,0,2.08,41.41,7,weekend
4,1,2013-02-20,1,0,2.08,41.41,3,workday


In [11]:
LE.fit(datos["Type.Day"])
list(LE.classes_)

['weekend', 'workday']

In [12]:
datos["Type.Day"] = LE.transform(datos["Type.Day"])
datos["Type.Day"].value_counts()

1    591925
0    232662
Name: Type.Day, dtype: int64

In [13]:
festivos = ('1-1','6-1','19-4','22-4','1-05','10-6','24-6','15-8','11-9','24-9','12-10','1-11','6-12','8-12','25-12','26-12')

festive = []
for fecha in datos['date']:
    pasat = False
    for festivo in festivos:
        aux = festivo.split("-")
        aux2 = datetime.strftime(fecha, '%Y-%m-%d').split("-")
        if ( (aux2[1] == aux[1]) and (aux2[2] == aux[0]) and (pasat == False) ):
            festive.append(1)
            pasat = True
    if (pasat == False):
        festive.append(0)

datos['Festive'] = festive
del festive
del festivos

datos.loc[datos['Week.Day'] > 6, 'Festive'] = 1 

In [14]:
datos['Festive'].value_counts()

0    703431
1    121156
Name: Festive, dtype: int64

In [15]:
datos.head()

,GridID,date,Shift,Accident,Longitude.grid,Latitude.grid,Week.Day,Type.Day,Festive
0,1,2010-10-08,2,0,2.08,41.41,5,1,0
1,1,2011-02-16,1,0,2.08,41.41,3,1,0
2,1,2014-05-31,2,0,2.08,41.41,6,0,0
3,1,2011-04-03,0,0,2.08,41.41,7,0,1
4,1,2013-02-20,1,0,2.08,41.41,3,1,0


Árbol decisiones

In [16]:
X = datos.drop("Accident",1)
X = X.drop("date",1)

In [17]:
y = datos["Accident"]

In [18]:
print(X.shape)
print(y.shape)

(824587, 7)
(824587,)


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.30, random_state=1)

In [20]:
print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)

(577210, 7)
(247377, 7)
(577210,)
(247377,)


In [21]:
from sklearn.tree import DecisionTreeClassifier
DTC = DecisionTreeClassifier(criterion='entropy', random_state=1)

In [22]:
DTC.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

In [23]:
y_train_pred = DTC.predict_proba(X_train)

In [24]:
from sklearn.metrics import roc_auc_score

In [25]:
print(roc_auc_score(y_train, y_train_pred[:, 1]), "Árbol de decisión - Datos de entrenamiento")

0.8842975712086046 Árbol de decisión - Datos de entrenamiento


In [26]:
y_test_pred = DTC.predict_proba(X_test)

In [27]:
print(roc_auc_score(y_test, y_test_pred[:, 1]), "Árbol de decisión - Datos de test")

0.7319843960335785 Árbol de decisión - Datos de test


# Ensamble

In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier

In [35]:
X = datos.drop("Accident",1)
X = X.drop("date",1)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.30, random_state=1)

In [37]:
DTC = DecisionTreeClassifier(random_state=1)

In [38]:
print("Estimación del set de Validacion usando DTC")
print(cross_val_score(DTC, X_train, y_train, cv=5, scoring='roc_auc').mean())

Estimación del set de Validacion usando DTC
0.7186310153929213


In [39]:
DTC.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

In [40]:
y_test_pred = DTC.predict_proba(X_test)

In [41]:
print("Metrica AUC del set Test usando DTC")
print(roc_auc_score(y_test, y_test_pred[:, 1]))

Metrica AUC del set Test usando DTC
0.7320152373104771


# Bagging

In [42]:
BAGG = BaggingClassifier(base_estimator=DTC, random_state=1)

In [43]:
print("Estimación del set de Validacion usando BAGG")
print(cross_val_score(BAGG, X_train, y_train, cv=5, scoring='roc_auc').mean())

Estimación del set de Validacion usando BAGG
0.7172323424846184


In [44]:
BAGG.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=None, oob_score=False,
         random_state=1, verbose=0, warm_start=False)

In [45]:
y_test_pred = BAGG.predict_proba(X_test)

In [46]:
print("Metrica AUC del set de Test usando BAGG")
print(roc_auc_score(y_test, y_test_pred[:, 1]))

Metrica AUC del set de Test usando BAGG
0.7312128538800423


# Boosting

In [47]:
from sklearn.ensemble import AdaBoostClassifier

In [48]:
BOOS = AdaBoostClassifier(random_state=1)

In [49]:
BOOS.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=1)

In [50]:
y_test_pred = BOOS.predict_proba(X_test)

In [51]:
print("Metrica AUC del set de Test usando BOOS")
print(roc_auc_score(y_test, y_test_pred[:, 1]))

Metrica AUC del set de Test usando BOOS
0.7544028697866263


# Voting

In [52]:
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [53]:
KNN = KNeighborsClassifier(n_neighbors=2)

In [54]:
LR = LogisticRegression(random_state=1)

In [55]:
VOTI = VotingClassifier(estimators=[("DTC", DTC), ("LR", LR), ("KNN", KNN)], voting='soft')

In [56]:
VOTI.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


VotingClassifier(estimators=[('DTC', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_lea...ki',
           metric_params=None, n_jobs=None, n_neighbors=2, p=2,
           weights='uniform'))],
         flatten_transform=None, n_jobs=None, voting='soft', weights=None)

In [57]:
y_test_pred = VOTI.predict_proba(X_test)

In [58]:
print("Metrica AUC del set de Test usando Voting")
print(roc_auc_score(y_test, y_test_pred[:, 1]))

Metrica AUC del set de Test usando Voting
0.7435058994390898


# Representación gráfica

Instalamos folium en terminal -> pip install folium

In [369]:
import folium

In [370]:
m = folium.Map(location=[41.38, 2.16],  tiles='Stamen Terrain', zoom_start=12)

In [371]:
m

In [372]:
m.save("barcelona.html")

In [373]:
datosM = datos

In [374]:
datosM.head()

,GridID,date,Shift,Accident,Longitude.grid,Latitude.grid,Week.Day,Type.Day,Festive
0,1,2010-10-08,2,0,2.08,41.41,5,1,0
1,1,2011-02-16,1,0,2.08,41.41,3,1,0
2,1,2014-05-31,2,0,2.08,41.41,6,0,0
3,1,2011-04-03,0,0,2.08,41.41,7,0,1
4,1,2013-02-20,1,0,2.08,41.41,3,1,0


In [375]:
noche = datosM[(datosM['Accident'] == 1) & (datosM['Shift'] == 2)]
manyana = datosM[(datosM['Accident'] == 1) & (datosM['Shift'] == 1)]
tarde = datosM[(datosM['Accident'] == 1) & (datosM['Shift'] == 0)]

In [376]:
print("Total noche", len(noche))
print("Total mañana",len(manyana))
print("Total tarde",len(tarde))

Total noche 4236
Total mañana 14375
Total tarde 17906


In [377]:
noche.rename(columns={'Longitude.grid': 'Longitude', 'Latitude.grid': 'Latitude'}, inplace=True)
manyana.rename(columns={'Longitude.grid': 'Longitude', 'Latitude.grid': 'Latitude'}, inplace=True)
tarde.rename(columns={'Longitude.grid': 'Longitude', 'Latitude.grid': 'Latitude'}, inplace=True)
nocheMapa = noche.sample(n=300)
manyanaMapa = manyana.sample(n=300)
tardeMapa = tarde.sample(n=300)

In [378]:
for indice_fila, fila in nocheMapa.iterrows():
    folium.Marker([fila['Latitude'], fila['Longitude']], 
                  popup='Accidente Noche', tooltip='Accidente', 
                  icon=folium.Icon(color="gray")).add_to(m)

for indice_fila, fila in manyanaMapa.iterrows():
    folium.Marker([fila['Latitude'], fila['Longitude']], 
                  popup='Accidente Mañana', tooltip='Accidente', 
                  icon=folium.Icon(color="red")).add_to(m)    

for indice_fila, fila in tardeMapa.iterrows():
    folium.Marker([fila['Latitude'], fila['Longitude']], 
                  popup='Accidente Tarde', tooltip='Accidente', 
                  icon=folium.Icon(color="blue")).add_to(m) 
    

In [379]:
m

In [380]:
m.save("barcelona.html")